# Load

Final cleaned merged dataset is loaded into gold folder.

## Configuration

In [23]:
import os
from google.colab import drive
# Check if Google Drive is already mounted, mount otherwise
if not os.path.ismount('/content/drive'):
    print("Google Drive is not mounted yet. Mounting...")
    drive.mount('/content/drive')
else:
    print("Google Drive is already mounted.")

Google Drive is already mounted.


In [24]:
import sys
sys.path.append('/content/drive/MyDrive/DataLife/modules')
import utils

In [25]:
#import re
#import glob
import pandas as pd
#from datetime import datetime

In [26]:
silver_path = "/content/drive/MyDrive/DataLife/data/silver"
gold_path = "/content/drive/MyDrive/DataLife/data/gold"

airport_silver_path = f"{silver_path}/fact_airport_traffic.csv"
gdp_silver_path = f"{silver_path}/fact_gdp_worldbank.csv"

## Airport Traffic

In [27]:
df_flights = pd.read_csv(airport_silver_path)

In [28]:
# Count unique values in column STATE_NAME
num_unique_countries = df_flights['STATE_NAME'].nunique()
print(f"Number of unique countries: {num_unique_countries}")

Number of unique countries: 43


In [29]:
# Get all unique countries and years from df_flights_final
all_countries = df_flights['STATE_NAME'].unique()
all_years = df_flights['YEAR'].unique()

# Create a complete set of all expected country-year combinations
from itertools import product
expected_combinations = set(product(all_countries, all_years))

# Create a set of existing country-year combinations from df_flights
existing_combinations = set(zip(df_flights['STATE_NAME'], df_flights['YEAR']))

# Find the missing combinations
missing_combinations = sorted(list(expected_combinations - existing_combinations))

if missing_combinations:
    print(f"Missing GDP data for the following country-year combinations ({len(missing_combinations)} missing):")
    for country, year in missing_combinations:
        print(f"  - Country: {country}, Year: {year}")
else:
    print("No missing GDP data found for any country-year combination in the current dataset.")

Missing GDP data for the following country-year combinations (14 missing):
  - Country: Iceland, Year: 2016
  - Country: Iceland, Year: 2017
  - Country: Iceland, Year: 2018
  - Country: Iceland, Year: 2019
  - Country: Iceland, Year: 2020
  - Country: Iceland, Year: 2021
  - Country: Iceland, Year: 2022
  - Country: Iceland, Year: 2023
  - Country: Israel, Year: 2016
  - Country: Israel, Year: 2017
  - Country: Morocco, Year: 2016
  - Country: Morocco, Year: 2017
  - Country: Ukraine, Year: 2024
  - Country: Ukraine, Year: 2025


In [30]:
# Delete countries from which we have missing information
countries_to_exclude = ['Iceland', 'Israel', 'Morocco', 'Ukraine']
df_flights = df_flights[~df_flights['STATE_NAME'].isin(countries_to_exclude)]

In [31]:
# Filter only required columns for our analysis
df_flights_filtered = df_flights[['YEAR','STATE_NAME','FLT_TOT_1']]

In [32]:
# Aggregate total of flights for Country and Year
df_flights_agg = df_flights_filtered.groupby(['STATE_NAME', 'YEAR']).agg({'FLT_TOT_1': 'sum'}).reset_index()

In [33]:
# Standarize names of columns and rename Turkey
df_flights_final = df_flights_agg.rename(columns={'STATE_NAME': 'country', 'YEAR': 'year', 'FLT_TOT_1': 'total_flights'}).replace({"Türkiye":"Turkiye"})

## GDP Worldbank

In [34]:
df_gdp = pd.read_csv(gdp_silver_path)

In [35]:
# Check if countries have different names on both datasets
countries_gdp = set(df_gdp['Country Name'].unique())
countries_flights = set(df_flights_final['country'].unique())
print(f"Countries in Flights dataset but not in GDP dataset: {countries_flights - countries_gdp}")

Countries in Flights dataset but not in GDP dataset: {'Czech Republic', 'Republic of North Macedonia', 'Slovakia'}


In [36]:
# Standardize country names for joins; by default, we will use the naming convention from the flights dataset.
country_name_mapping = {
    'Czechia': 'Czech Republic',
    'North Macedonia': 'Republic of North Macedonia',
    'Slovak Republic': 'Slovakia'
}
df_gdp['Country Name'] = df_gdp['Country Name'].replace(country_name_mapping)

# Re-check the country differences after replacement
countries_gdp = set(df_gdp['Country Name'].unique())
countries_flights = set(df_flights_final['country'].unique())

print(f"Countries in Flights but not in GDP: {countries_flights - countries_gdp}")

Countries in Flights but not in GDP: set()


In [37]:
# Filter only by European Countries from whose we have information about flights.
df_gdp_filtered = df_gdp[df_gdp['Country Name'].isin(countries_flights)]

In [38]:
# Melt dataframe to copy structure from flights dataset
df_gdp_melted = df_gdp_filtered.melt(id_vars=['Country Name'], var_name='year', value_name='gdp')

# Convert 'year' column to integer type
df_gdp_melted['year'] = df_gdp_melted['year'].astype(int)

In [39]:
# Rename 'Country Name' to 'country'
df_gdp_final = df_gdp_melted.rename(columns={'Country Name': 'country'})

## Final Join

In [40]:
# Merge the two DataFrames on 'country' and 'year'
df_final = pd.merge(df_flights_final, df_gdp_final, on=['country', 'year'], how='inner')

In [41]:
utils.df_to_csv(df_final, f"final_dataset.csv", gold_path)

File correctly saved at: /content/drive/MyDrive/DataLife/data/gold/final_dataset.csv
